## Implementation

La mise en œuvre du projet comporte plusieurs étapes, notamment la configuration des bibliothèques nécessaires, l'initialisation de la solution MediaPipe hand, la définition des détails des formes et des boutons et l'implémentation de la boucle principale qui traite chaque image provenant de la webcam. Voici une décomposition détaillée du code :

### Importation des bibliothèques nécessaires

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time

Le projet utilise quatre bibliothèques : OpenCV (cv2) pour le traitement d'images et la capture de vidéos à partir de la webcam, MediaPipe (mp) pour la détection des points de repère de la main, NumPy (np) pour les opérations numériques, et time pour les opérations de chronométrage.

### Initialisation de la solution MediaPipe Hand

In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(model_complexity=0, min_detection_confidence=0.5, min_tracking_confidence=0.5)

La solution main de MediaPipe est initialisée avec une complexité de modèle de 0 (le modèle le plus rapide mais le moins précis), et une confiance minimale de détection et de suivi de 0,5.

### Définition de la forme et des détails du bouton

In [ ]:
# rectangle and circle details
rect_width, rect_height = 100, 100
coord_rectangle_1 = [(300 - rect_width // 2, 300 - rect_height // 2), (300 + rect_width // 2, 300 + rect_height // 2)]
coord_rectangle_2 = [(400 - rect_width // 2, 400 - rect_height // 2), (400 + rect_width // 2, 400 + rect_height // 2)]
coord_circle = [(200, 200), 50]  # center and radius of the circle

Les détails des formes (rectangles et cercles) qui seront dessinées à l'écran sont définis. Les coordonnées des rectangles et du cercle sont définies, ainsi que la largeur et la hauteur des rectangles et le rayon du cercle.


In [ ]:
# Add Rect and Add Circle button details
button_width, button_height = 50, 50
coord_add_rect_button = [(50 - button_width // 2, 50 - button_height // 2), (50 + button_width // 2, 50 + button_height // 2)]
coord_add_circle_button = [(100 + button_width // 2, 50 - button_height // 2), (150 + button_width // 2, 50 + button_height // 2)]
coord_remove_button = [(450 - button_width // 2, 450 - button_height // 2), (450 + button_width // 2, 450 + button_height // 2)]
coord_exit_button = [(550 - button_width // 2, 50 - button_height // 2), (600 + button_width // 2, 50 + button_height // 2)]

Les détails des boutons (Ajouter un rectangle, Ajouter un cercle, Supprimer et Quitter) qui seront dessinés à l'écran sont définis. Les coordonnées des boutons sont définies, ainsi que leur largeur et leur hauteur.

### Boucle principale

In [ ]:
try:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

La boucle principale du programme commence par vérifier si la webcam est ouverte. Si c'est le cas, elle lit une image de la webcam. Si la trame est vide, il saute le reste de la boucle et passe à l'itération suivante.

In [ ]:
        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

L'image est retournée horizontalement pour créer un effet miroir, et elle est convertie de l'espace colorimétrique BGR (utilisé par OpenCV) à l'espace colorimétrique RVB (utilisé par MediaPipe).

In [ ]:
        # process the image to find hand landmarks
        image.flags.writeable = False
        results = hands.process(image)

L'image est traitée par la solution main de MediaPipe pour trouver les points de repère de la main. L'attribut 'flags.writeable' de l'image est fixé à False pour empêcher MediaPipe de modifier l'image.

In [ ]:
        # draw the hand landmarks on the image
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

Les points de repère de la main et les connexions entre eux sont dessinés sur l'image. L'attribut "flags.writeable" de l'image est remis à True pour permettre les modifications, et l'image est reconvertie dans l'espace colorimétrique BGR pour être affichée avec OpenCV.

In [ ]:
                # get coordinates of thumb tip and index finger tip
                x_thumb = int(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x * image.shape[1])
                y_thumb = int(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y * image.shape[0])
                x_index = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image.shape[1])
                y_index = int(hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image.shape[0])

Les coordonnées de l'extrémité du pouce et de l'index sont extraites des repères de la main. Ces coordonnées sont normalisées en fonction de la taille de l'image.

In [ ]:
                # calculate distance between thumb tip and index finger tip
                distance = np.sqrt((x_thumb - x_index) ** 2 + (y_thumb - y_index) ** 2)

La distance entre l'extrémité du pouce et l'extrémité de l'index est calculée à l'aide de la formule de la distance euclidienne. Cette distance permet de déterminer si l'utilisateur fait un geste de pincement.

In [ ]:
                # draw and interact with rectangles
                for i, rect in enumerate(rectangles):
                    if (rect[0][0] < x_index < rect[1][0] and rect[0][1] < y_index < rect[1][1]) and (distance < threshold_distance):
                        cv2.rectangle(image, rect[0], rect[1], new_color, 2)
                        rect[0], rect[1] = (x_index - rect_width // 2, y_index - rect_height // 2), (x_index + rect_width // 2, y_index + rect_height // 2)
                        if coord_remove_button[0][0] < x_index < coord_remove_button[1][0] and coord_remove_button[0][1] < y_index < coord_remove_button[1][1]:
                            del rectangles[i]
                    else:
                        cv2.rectangle(image, rect[0], rect[1], default_color, 2)

The rectangles are drawn on the image. If the index finger tip is inside a rectangle and the user is making a pinching gesture, the rectangle is moved to the position of the index finger tip. If the index finger tip is also inside the Remove button, the rectangle is removed.

In [ ]:
                # draw and interact with circles
                for i,

 circle in enumerate(circles):
                    if (circle[0][0] - circle[1] < x_index < circle[0][0] + circle[1] and circle[0][1] - circle[1] < y_index < circle[0][1] + circle[1]) and (distance < threshold_distance):
                        cv2.circle(image, circle[0], circle[1], new_color, 2)
                        circle[0] = x_index, y_index
                        if coord_remove_button[0][0] < x_index < coord_remove_button[1][0] and coord_remove_button[0][1] < y_index < coord_remove_button[1][1]:
                            del circles[i]
                    else:
                        cv2.circle(image, circle[0], circle[1], default_color, 2)

Les cercles sont dessinés sur l'image. Si le bout de l'index se trouve à l'intérieur d'un cercle et que l'utilisateur fait un geste de pincement, le cercle est déplacé à la position du bout de l'index. Si le bout de l'index se trouve également à l'intérieur du bouton Supprimer, le cercle est supprimé.

In [ ]:
                # draw the "Add Rect" button
                if coord_add_rect_button[0][0] < x_index < coord_add_rect_button[1][0] and coord_add_rect_button[0][1] < y_index < coord_add_rect_button[1][1]:
                    cv2.rectangle(image, coord_add_rect_button[0], coord_add_rect_button[1], new_color, 2)
                    cv2.putText(image, "Add Rect", (coord_add_rect_button[0][0] - 10, coord_add_rect_button[0][1] - 10), font, font_scale, font_color, line_type)
                    if start_time_rect is None:
                        start_time_rect = time.time()
                    elif time.time() - start_time_rect >= click_duration:
                        # add new rectangle
                        new_rect = [(400 - rect_width // 2, 400 - rect_height // 2), (400 + rect_width // 2, 400 + rect_height // 2)]
                        rectangles.append(new_rect)
                        start_time_rect = None
                else:
                    cv2.rectangle(image, coord_add_rect_button[0], coord_add_rect_button[1], button_color, 2)
                    cv2.putText(image, "Add Rect", (coord_add_rect_button[0][0] - 10, coord_add_rect_button[0][1] - 10), font, font_scale, font_color, line_type)
                    start_time_rect = None

Le bouton Ajouter un rectangle est dessiné sur l'image. Si le bout de l'index se trouve à l'intérieur du bouton, celui-ci change de couleur et si le bout de l'index reste à l'intérieur du bouton pendant un certain temps, un nouveau rectangle est ajouté.

In [ ]:
                # draw the "Add Circle" button
                if coord_add_circle_button[0][0] < x_index < coord_add_circle_button[1][0] and coord_add_circle_button[0][1] < y_index < coord_add_circle_button[1][1]:
                    cv2.rectangle(image, coord_add_circle_button[0], coord_add_circle_button[1], new_color, 2)
                    cv2.putText(image, "Add Circle", (coord_add_circle_button[0][0] - 10, coord_add_circle_button[0][1] - 10), font, font_scale, font_color, line_type)
                    if start_time_circle is None:
                        start_time_circle = time.time()
                    elif time.time() - start_time_circle >= click_duration:
                        # add new circle
                        new_circle = [(200, 200), 50]
                        circles.append(new_circle)
                        start_time_circle = None
                else:
                    cv2.rectangle(image, coord_add_circle_button[0], coord_add_circle_button[1], button_color, 2)
                    cv2.putText(image, "Add Circle", (coord_add_circle_button[0][0] - 10, coord_add_circle_button[0][1] - 10), font, font_scale, font_color, line_type)
                    start_time_circle = None

Le bouton Ajouter un cercle est dessiné sur l'image. Si le bout de l'index se trouve à l'intérieur du bouton, celui-ci change de couleur et si le bout de l'index reste à l'intérieur du bouton pendant un certain temps, un nouveau cercle est ajouté.


In [ ]:
                # draw the "Remove" button
                cv2.rectangle(image, coord_remove_button[0], coord_remove_button[1], button_color, 2)
                cv2.putText(image, "Remove", (coord_remove_button[0][0] - 10, coord_remove_button[0][1] - 10), font, font_scale, font_color, line_type)

Le bouton Supprimer est dessiné sur l'image. Si le bout de l'index se trouve à l'intérieur d'une forme et que la forme se trouve à l'intérieur du bouton Supprimer, la forme est supprimée.

In [ ]:
                # draw the "Exit" button
                if coord_exit_button[0][0] < x_index < coord_exit_button[1][0] and coord_exit_button[0][1] < y_index < coord_exit_button[1][1]:
                    cv2.rectangle(image, coord_exit_button[0], coord_exit_button[1], new_color, 2)
                    cv2.putText(image, "Exit", (coord_exit_button[0][0] - 10, coord_exit_button[0][1] - 10), font, font_scale, font_color, line_type)
                    if start_time_exit is None:
                        start_time_exit = time.time()
                else:
                    cv2.rectangle(image, coord_exit_button[0], coord_exit_button[1], button_color, 2)
                    cv2.putText(image, "Exit", (coord_exit_button[0][0] - 10, coord_exit_button[0][1] - 10), font, font_scale, font_color, line_type)
                    start_time_exit = None

Le bouton Quitter est dessiné sur l'image. Si le bout de l'index se trouve à l'intérieur du bouton, celui-ci change de couleur, et si le bout de l'index reste à l'intérieur du bouton pendant un certain temps, le programme se termine.

In [ ]:
            # check if the exit button is pressed
            if start_time

_exit is not None and time.time() - start_time_exit >= click_duration:
                break

Le programme vérifie si le bouton Exit est enfoncé. Si c'est le cas, la boucle principale est quittée et le programme se termine.


In [ ]:
        cv2.imshow('MediaPipe Hands', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()

L'image traitée est affichée dans une fenêtre. Si la touche 'q' est pressée, la boucle principale est quittée et le programme se termine. Après la boucle principale, la webcam est libérée et toutes les fenêtres sont détruites.